# Natural Language Inference using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

In [1]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext
import torch.optim as optim
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
import numpy as np

# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:3')

batch_size = 4
learning_rate = 0.001
epochs = 3
max_grad_norm = 5

In this lab we'll work with neural networks for natural language inference. Our task is: given a premise sentence P and hypothesis H, what entailment relationship holds between them? Is H entailed by P, contradicted by P or neutral towards P?

Given a sentence P, if H definitely describe something true given P then it is an **entailment**. If H describe something that's *maybe* true given P, it's **neutral**, and if H describe something that's definitely *false* given P it's a **contradiction**. 

# 1. Data

We will explore natural language inference using neural networks on the SNLI dataset, described in [1]. The dataset can be downloaded [here](https://nlp.stanford.edu/projects/snli/). We prepared a "simplified" version, with only the relevant columns [here](https://gubox.box.com/s/idd9b9cfbks4dnhznps0gjgbnrzsvfs4).

The (simplified) data is organized as follows (tab-separated values):
* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

Like in the previous lab, we'll use torchtext to build a dataloader. You can essentially do the same thing as you did in the last lab, but with our new dataset. **[1 mark]**

In [2]:
def dataloader(path_to_snli):
        
    whitespacer = lambda x: x.split(' ')

    # "fields" that process the different columns in our CSV files
    PREMISE = Field(tokenize    = whitespacer,
               lower       = True,
               batch_first = True)

    HYPOTHESIS = Field(tokenize    = whitespacer,
                  lower       = True,
                  batch_first = True)
    
    RELATION = Field(tokenize    = whitespacer,
                     sequential = False,
                     use_vocab = False, # To make sure you don't use the vocabulary for this field
                     batch_first = True)
    
    # read the csv files
    train, val, test = TabularDataset.splits(path   = path_to_snli,
                                        train  = 'simple_snli_1.0_train.csv',
                                        validation = 'simple_snli_1.0_dev.csv',
                                        test   = 'simple_snli_1.0_test.csv',
                                        format = 'csv',
                                        fields = [('premise', PREMISE),
                                                  ('hypothesis', HYPOTHESIS),
                                                 ('relation', RELATION)],
                                        skip_header       = True,
                                        csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'½'})
    
    # build vocabularies based on what our csv files contained and create word2id mapping
    PREMISE.build_vocab(train)
    HYPOTHESIS.build_vocab(train)

    # create batches from our data, and shuffle them for each epoch
    train_iter, val_iter, test_iter = BucketIterator.splits((train, val, test),
                                                  batch_size        = batch_size,
                                                  sort_within_batch = True,
                                                  sort_key          = lambda x: len(x.premise),
                                                  shuffle           = True,
                                                  device            = device)

    return train_iter, val_iter, test_iter, PREMISE, HYPOTHESIS, RELATION

# 2. Model

In this part, we'll build the model for predicting the relationship between H and P.

We will **process each sentence** using an LSTM. Then, we will **construct some representation of the sentence**. When we have a representation for H and P, we will **combine them into one vector** which we can use to predict the relationship.

We will train a model described in [2], the BiLSTM with max-pooling model. The procedure for the model is roughly:

    1) Encode the Hypothesis and the Premise using one shared bidirectional LSTM (or two different LSTMS)
    2) Perform max over the tokens in the premise and the hypothesis
    3) Combine the encoded premise and encoded hypothesis into one representation
    4) Predict the relationship 

### Creating a representation of a sentence

Let's first consider step 2 where we perform max/mean pooling. There is a function in pytorch for this, but we'll implement it from scratch. 

Let's consider the general case, what we want to do for these methods is apply some function $f$ along dimension $i$, and we want to do this for all $i$'s. As an example we consider the matrix S with size ``(N, D)`` where N is the number of words and D the number of dimensions:

$S = \begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1d} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{n1} & s_{n2} & s_{n3} & \dots  & s_{nd}
\end{bmatrix}$

What we want to do is apply our function $f$ on each dimension, taking the input $s_{1d}, s_{2d}, ..., s_{nd}$ and generating the output $x_d$. 

You will implement both the max pooling method. When performing max-pooling, $max$ will be the function which selects a _maximum_ value from a vector and $x$ is the output, thus for each dimension $d$ in our output $x$ we get:

\begin{equation}
    x_d = max(s_{1d}, s_{2d}, ..., s_{nd})
\end{equation}


This operation will reduce a batch of size ``(batch_size, num_words, dimensions)`` to ``(batch_size, dimensions)`` meaning that we now have created a sentence representation based on the content of the words representations in the sentence. 

Create a function that takes as input a tensor of size ``(batch_size, num_words, dimensions)`` then performs max pooling and returns the result (the output should be of size: ```(batch_size, dimensions)```). [**4 Marks**]

In [4]:
def pooling(input_tensor):
    l = []
    for x in torch.transpose(input_tensor,2,1):
        m = []
        for y in x:
            n = float(y[0])
            for z in y:
                if float(z)>n:
                    n = float(z)
            m.append(n)
        l.append(m)
    output_tensor = torch.tensor(l)
    return output_tensor

In [5]:
a = torch.randn(2, 5, 7)
a # pooling(a) should return something of size (2, 7)

tensor([[[-1.5600e+00, -1.4349e+00, -6.9112e-03, -8.3449e-01, -7.9779e-01,
          -5.2512e-01,  9.6264e-01],
         [ 5.5630e-01,  3.1895e-01,  2.4964e-01, -2.6750e-01, -1.0591e+00,
           1.3185e+00, -2.1923e-03],
         [ 1.5568e+00, -3.6314e-02,  5.8799e-01,  2.4178e-01, -1.6537e-01,
           8.7201e-01,  1.6189e+00],
         [ 2.8835e+00, -6.6424e-01,  1.0377e+00,  8.1184e-01, -1.5622e+00,
           1.2081e-01,  4.6629e-01],
         [ 9.1862e-01,  1.4051e+00,  3.5791e-01,  1.1596e+00,  9.3952e-01,
          -9.5961e-02,  3.8685e-01]],

        [[-1.4309e+00, -3.3521e-01,  7.3643e-02, -1.1282e+00, -2.8372e-01,
          -7.0371e-01,  8.5056e-01],
         [-1.1593e+00, -7.9697e-01, -1.7846e+00, -2.0383e-01,  6.5730e-01,
          -1.0690e-01, -6.0873e-01],
         [ 2.3997e-01, -1.8274e+00,  7.4123e-01, -1.3074e+00, -6.6188e-01,
           1.5690e+00, -9.6435e-01],
         [-9.7739e-01, -5.0703e-01, -1.2722e+00,  7.3621e-01, -9.0196e-02,
           9.1219e-02, -2.2

In [6]:
b = pooling(a)
print(b, b.size())

tensor([[ 2.8835,  1.4051,  1.0377,  1.1596,  0.9395,  1.3185,  1.6189],
        [ 0.2400, -0.1661,  0.7412,  0.8456,  0.7867,  1.5690,  0.8506]]) torch.Size([2, 7])


In [13]:
assert (torch.max(a,1).values == pooling(a)).all() # it works! (づ￣ 3￣)づ

### Combining sentence representations

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

Implement the function. **[2 marks]**

In [14]:
def combine_premise_and_hypothesis(hypothesis, premise):
    minus = torch.abs(premise - hypothesis)
    times = torch.mul(premise, hypothesis)
    output = torch.cat((premise, hypothesis, minus, times), dim = 1)
    return output

In [15]:
c = torch.randn(2, 5, 7)
d = pooling(c)
combine_premise_and_hypothesis(b, d).size()

torch.Size([2, 28])

### Creating the model

Finally, we can build the model according to the procedure given previously by using the functions we defined above. Additionaly, in the model you should use *dropout*. For efficiency purposes, it's acceptable to only train the model with either max or mean pooling. 

Implement the model [**6 marks**]

In [ ]:
class SNLIModel(nn.Module):
    def __init__(self, num_words, i_dim, o_dim):
        super(SNLIModel, self).__init__()
        self.embeddings = nn.Embedding(num_words, i_dim)
        self.rnn = nn.LSTM(i_dim, o_dim, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(o_dim*2, 3) # 3 is the number of relations: E, N, C; but I'm not sure if it should go here
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, premise, hypothesis):
        p = self.embeddings(premise)
        h = self.embeddings(hypothesis)
        
        p_pooled = pooling(p)
        h_pooled = pooling(h)
        
        ph_representation = combine_premise_and_hypothesis(h, p) # Be careful with the stupid order of the arguments!
        
        hidden_states, (final_hidden, cell_state) = self.rnn(ph_representation)
        hidden_states = self.dropout(hidden_states)
        
        predictions = self.classifier(hidden_states) # I might be missing some indexing
        
        return predictions

# 3. Training and testing

As before, implement the training and testing of the model. SNLI can take a very long time to train, so I suggest you only run it for one or two epochs. **[2 marks]** 

**Tip for efficiency:** *when developing your model, try training and testing the model on one batch (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [3]:
path_to_snli = "../../../05-natural-language-inference/data/"
train_iter, dev_iter, test_iter, premise, hypothesis, relation = dataloader(path_to_snli)

In [ ]:
# train_iter, test_iter = dataloader(path_to_folder)

num_words = # the union of premise.vocab and hypothesis.vocab?

loss_function = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model = SNLIModel(num_words, i_dim, o_dim)

model.train()

for _ in range(epochs):
    # train model
    ...
    
# test model after all epochs are completed

Suggest a _baseline_ that we can compare our model against **[2 marks]**

**Your answer should go here**

Suggest some ways (other than using a baseline) in which we can analyse the models performance **[4 marks]**.

**Your answer should go here**

Suggest some ways to improve the model **[3 marks]**.

**Your answer should go here**

### Readings

[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.